# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [1]:
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add

Using TensorFlow backend.
C:\Users\a2541\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\a2541\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\a2541\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\a2541\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

##### 定義 Separable Convolution 函數 (請在此實做)

In [2]:
def InvertedRes(input, expansion):
    '''
    Args:
        input: input tensor
        expansion: expand filters size
    Output:
        output: output tensor
    '''
    # Expansion Layer
    x = Conv2D(expansion * 3, (1,1), padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Depthwise Convolution
    x = DepthwiseConv2D((3,3), padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Pointwise Convolution
    x = Conv2D(3, (1,1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Add Layer
    x = Add()([input, x])
    
    return x

##### 建構模型

In [3]:
input_ = Input((64, 64, 3))
output = InvertedRes(input_, 6)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
depthwise_conv2d_1 (DepthwiseCo (None, 64, 64, 3)    30          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 3)    12          depthwise_conv2d_1[0][0]         
__________________________________________________________________________________________________
re_lu_2 (ReLU)                  (None, 64, 64, 3)    0           batch_normalization_2[0][0]      
____________________________________________________________________________________________